<a href="https://colab.research.google.com/github/adekunleba/tensorflow_tutorial/blob/master/Linear_Regression_with_Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Linear Regression with Tensorflow Eeager Execution

In [0]:
import tensorflow as tf
tf.enable_eager_execution()

Every machine learning model application starts with establishing the data, we can make use of a synthetic data as a starting point.

Also, we will also train a model on one other real data, the aim is to show the numerical computation capacity of Tensorflow.

In [0]:
def make_synthetic_data(w, b, noise_level, batch_size, num_batches):
  def batch(_):
    x = tf.random_normal(shape=[batch_size, tf.shape(w)[0]])
    y = tf.matmul(x, w) + b + noise_level + tf.random_normal([])
    return x, y
  
  
  #Use Cpu to do this
  with tf.device("/device:CPU:0"):
    return tf.data.Dataset.range(num_batches).map(batch)

Make a synthetic data using the `make_synthetic_data` function, following this, you give a base weight and bias value for you model to be optimized on.



In [0]:
true_w = [[-2.0], [4.0], [1.0]]
true_b = [0.5]
noise_level = 0.01

# Training constants.
batch_size = 64
learning_rate = 0.1 




data = make_synthetic_data(true_w, true_b, noise_level, batch_size, 20)

In [0]:
layers = tf.keras.layers


model = layers.Dense(1)

In [0]:
mse = lambda xs, ys: tf.reduce_mean(tf.square(tf.subtract(model(xs), ys)))

Using `implicit_value_and_gradient` to generate a gradient function. Basically, the function takes another function as input, the input function is usually a computation that needs to undergo differentiation, say for example a *loss function*. 

`impliit_value_and_gradient` returns the values and gradient obtained upon differentiating an input function.

A possible replacement for the `mse lambda function` above will be something like this.


```python

def loss(xs, ys):
  return tf.reduce_mean(tf.square(tf.subtract(model(xs), ys)))
 ```

In [0]:
tfe = tf.contrib.eager
loss_and_grads = tfe.implicit_value_and_gradients(f=mse)

Optimizer is what helps to shape the model to it's optimum values by comparing the loss upon every forward pass through the network and updating the model parameters accordingly.

One key thing to note is how loss functions and optimizers work together to give the best possible model. The results from the loss functions helps the  optimization algorithm makes it's decision on how to change the various parameter to tend it towards the most accurate.

Usually depicted with the hiker's approach to find the lowest loss.

Hence you can start to thing of algorithms such as `gradient descent` in it's various forms i.e `stochastic` and `mini-batch`, others like `Adagrad`, `RMSprop` and `Adam` optimizers.

In [0]:
#Build Optimizer

optimizer = tf.train.GradientDescentOptimizer(learning_rate)

In [0]:
for i, (xs, ys) in enumerate(tfe.Iterator(data)):
  loss, grads = loss_and_grads(xs, ys)
  print("Iteration %d: loss = %s" % (i, loss.numpy()))
  
  optimizer.apply_gradients(grads)

Iteration 0: loss = 26.094215
Iteration 1: loss = 12.233289
Iteration 2: loss = 10.095221
Iteration 3: loss = 11.4293165
Iteration 4: loss = 5.6117835
Iteration 5: loss = 6.0150127
Iteration 6: loss = 3.2144234
Iteration 7: loss = 2.397113
Iteration 8: loss = 1.4619782
Iteration 9: loss = 0.8915719
Iteration 10: loss = 0.43009764
Iteration 11: loss = 1.1105876
Iteration 12: loss = 0.66838795
Iteration 13: loss = 0.32206523
Iteration 14: loss = 0.22130188
Iteration 15: loss = 0.4471932
Iteration 16: loss = 1.263875
Iteration 17: loss = 0.8236362
Iteration 18: loss = 0.8474743
Iteration 19: loss = 0.8903562


In [0]:
print("Model Weight After Training \n {}".format(model.variables[0].numpy()))

print("Model Bias After Training \n {}".format(model.variables[1].numpy()))

Model Weight After Training 
 [[-2.0070481 ]
 [ 3.9331315 ]
 [ 0.98577815]]
Model Bias After Training 
 [0.12417183]


***This is a later approach - Do functional approach first***

Since tensorflow uses keras model out of the box, hence our model will use the keras way of building custom models.

Hence we can structure our codes in a more organized approach.


There are basic 3 components for your custom model when using Tensorflow Eager Execution:


* init Method: Define your various networks.
* call method: Invokes the model, this is where the layer logic all resides. Most times, the sequential approach to modelling Keras models actually happen here.
* fit method: Method fit assembled model on data.
* predict method: Method predicts on new data, same as call, but keras Model subclassing requires that you implement call.

***Other Utility Functions***

* Save model method
* compute_accuracy
* restore_model


In [0]:
class LinearRegressionModel(tf.keras.Model):
  
  def __init__(self):
    super(LinearRegressionModel, self).__init__()
    
    #Construct a single dense layer which is actually a linear layer.
    self._layer = layers.Dense(1) #Creates model as seen above
    
    
  def call(self, xs):
    
    return self._layer(xs) # check up it's same thing as model(xs)
  
  
  
  
def fit(model, dataset, optimizer, verbose=False, logdir=None):

  ''' Defines the model activity on a dataset

  Args:

    model: A Tensorflow-Keras to fit
    dataset: the tf.data.Dataset to use as training data
    optimizer: The tensorflow optimizer to be used for model update
  '''

  #Check out the use of self.call here
  mse = lambda xs, ys: tf.reduce_mean(tf.square(tf.subtract(model(xs), ys)))
  loss_and_grads = tfe.implicit_value_and_gradients(mse)


  '''Make Summary Writer'''
  if logdir:
    summary_writer = tf.contrib.summary.create_file_writer(logdir)


  '''Train and get loss and grads on dataset'''
  for i, (xs, ys) in enumerate(tfe.Iterator(dataset)):
    loss, grads = loss_and_grads(xs, ys)

    if verbose:
      print("Iteration %d: loss = %s" % (i, loss.numpy()))

    '''Apply optimizer on gradients for every run over the dataset'''
    optimizer.apply_gradients(grads)


    '''Write to summary-writer'''
    if logdir:
      with summary_writer.as_default():
        with tf.contrib.summary.always_record_summaries():
          tf.contrib.summary.scala("loss", loss, step=i)
          tf.contrib.summary.scala("step", i, step= i)

In [0]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate)

In [0]:
model = LinearRegressionModel()
fit(model, data, optimizer,True)

Iteration 0: loss = 41.38832
Iteration 1: loss = 16.406015
Iteration 2: loss = 10.869165
Iteration 3: loss = 17.738844
Iteration 4: loss = 3.0250764
Iteration 5: loss = 2.1878157
Iteration 6: loss = 1.2366109
Iteration 7: loss = 0.871888
Iteration 8: loss = 0.60339653
Iteration 9: loss = 2.1640959
Iteration 10: loss = 0.77372444
Iteration 11: loss = 0.8446425
Iteration 12: loss = 0.15529142
Iteration 13: loss = 0.15771008
Iteration 14: loss = 2.320445
Iteration 15: loss = 0.4848066
Iteration 16: loss = 2.276996
Iteration 17: loss = 0.19357428
Iteration 18: loss = 0.14999917
Iteration 19: loss = 1.9393296


Further more, if you want to go the old keras way, you can extract your data as numpy array do a `model.compile` on your Custom Model, then do a `model.fit`.

Tensorflow standard approach is to build your custom model with only the `call` function and then use other functions to either save the model, run the fit model and many other things you might want to do with the model.

In [0]:
from sklearn.datasets import make_moons
import numpy as np


X, y = make_moons(n_samples=100, noise=0.1, random_state=2018)

In [0]:
 tf.constant(X)

<tf.Tensor: id=1, shape=(100, 2), dtype=float64, numpy=
array([[ 1.77190359e-01,  7.46478326e-02],
       [ 1.01924195e+00,  3.98295353e-01],
       [ 1.81463750e+00,  6.90291309e-02],
       [-7.66548767e-01,  4.08949954e-01],
       [ 1.94298358e+00,  1.83284645e-01],
       [-6.11728497e-01,  8.49077927e-01],
       [ 5.83747133e-01,  8.77707116e-01],
       [ 1.58144580e-02,  2.43299815e-01],
       [ 2.91769822e-02,  5.38482295e-01],
       [ 1.18874861e+00,  6.91382429e-02],
       [ 1.21413031e+00, -4.73840709e-01],
       [-6.37130161e-01,  6.41576418e-01],
       [ 1.10865115e+00,  4.28466554e-01],
       [ 2.49046492e-01, -2.65682656e-01],
       [ 8.67971835e-01, -6.39420726e-01],
       [ 2.03199442e+00,  3.50406817e-01],
       [ 8.55375327e-02,  5.44524908e-01],
       [ 1.99874540e+00,  6.12304969e-01],
       [ 5.71342365e-01, -2.84712726e-01],
       [-9.75279446e-01,  5.15445098e-01],
       [ 2.10165156e+00,  2.55667917e-01],
       [ 8.36251588e-02,  9.76239529e-01]